## Import lib.

In [2]:
import os
import zipfile


import cv2
import matplotlib.pyplot as plt 
import numpy as np 
import torch 
import torch.nn as nn 

## Import data

In [17]:
with open(".env") as f:
    for line in f:
        key, val = line.strip().split("=")
        os.environ[key] = val

# print("Kaggle user:", os.environ['KAGGLE_USERNAME'])  
# print("Kaggle user:", os.environ['KAGGLE_KEY'])  

In [18]:
!kaggle datasets download -d fantacher/neu-metal-surface-defects-data

Dataset URL: https://www.kaggle.com/datasets/fantacher/neu-metal-surface-defects-data
License(s): other




  0%|          | 0.00/57.4M [00:00<?, ?B/s]
100%|██████████| 57.4M/57.4M [00:00<00:00, 1.55GB/s]


In [19]:
with zipfile.ZipFile("neu-metal-surface-defects-data.zip", "r") as zip_ref:
    zip_ref.extractall("neu_defects")